In [ ]:

import pandas as pd
import numpy as np
import string
import itertools
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import joblib

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


ClickBait


In [ ]:
data=pd.read_csv('/content/clickbait_data.csv')
print(data.head(10))

                                            headline  clickbait
0                                 Should I Get Bings          1
1      Which TV Female Friend Group Do You Belong In          1
2  The New "Star Wars: The Force Awakens" Trailer...          1
3  This Vine Of New York On "Celebrity Big Brothe...          1
4  A Couple Did A Stunning Photo Shoot With Their...          1
5  How To Flirt With Queer Girls Without Making A...          1
6  32 Cute Things To Distract From Your Awkward T...          1
7             If Disney Princesses Were From Florida          1
8  What's A Quote Or Lyric That Best Describes Yo...          1
9  Natalie Dormer And Sam Claflin Play A Game To ...          1


In [ ]:
stop=set(stopwords.words('english'))
stop.update(list(string.punctuation))
print(stop)
print(data.head(5))


{'shouldn', '&', 'have', 'or', 'hasn', 'be', '>', '~', 'than', 'its', 'should', "didn't", 'in', "you've", "haven't", 'here', 'any', 'both', '\\', 'those', 'we', 've', 'himself', "she's", "that'll", 'few', 'theirs', 'not', 'off', 'more', 'against', 'weren', 'it', 'again', 'the', 'isn', '*', '<', "shouldn't", 'no', 'further', 'him', 'is', 'don', 'herself', 'as', 'ma', "hadn't", '"', '=', 'couldn', 're', 'ours', 'because', 'after', 'once', '(', '{', 'just', 'myself', 'our', 'ain', 'i', 'each', '}', 'had', 'mustn', 'yours', 'shan', 'what', 'but', 'itself', 'does', 'so', '.', 'from', '!', 'some', "mightn't", 'nor', 'won', '-', 'only', 'y', 'by', 'about', 'd', 'on', "aren't", 'o', '/', 'doesn', 'for', 'until', 'into', '#', '@', 'hers', "it's", 'whom', '+', 'he', 'of', 'if', "doesn't", 'an', 'to', 's', 't', 'mightn', '`', 'while', "isn't", "mustn't", 'needn', 'under', 'their', "couldn't", 'how', 'do', 'themselves', 'other', '_', '|', 'this', 'she', 'and', 'am', 'having', "don't", "should've",

Cleaning the Data by removing punctuations, Stopwords and Special Characters

In [ ]:
finaldata=pd.read_csv('/content/clickbait_data.csv')

#shuffle the rows
finaldata=finaldata.sample(frac=1)

#Removing Null Rows
finaldata.dropna(subset=['headline','clickbait'],inplace=True)

#Converting to lower case
finaldata['headline']=finaldata['headline'].str.lower()

#Filtering the data to remove stopwords and punctuation
for j in range(len(finaldata['headline'])):
  temp=''
  for i in finaldata['headline'][j].split():
    if i.strip() not in stop:
      temp=temp+" "+i.strip()
  finaldata['headline'][j]=temp

#Removing Special Characters
spec_chars = ["-","!",'"',"#","%","&","'","(",")","*","+",",","-",".","/",":",";","<","=",">","?","@","[","\\","]","^","_","`","{","|","}","~","–"]
for char in spec_chars:
    finaldata['headline'] = finaldata['headline'].str.replace(char, '')

print("Original File:")
print(data.head(5))
print("Filtered Data:")
print(finaldata.head(5))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Original File:
                                            headline  clickbait
0                                 Should I Get Bings          1
1      Which TV Female Friend Group Do You Belong In          1
2  The New "Star Wars: The Force Awakens" Trailer...          1
3  This Vine Of New York On "Celebrity Big Brothe...          1
4  A Couple Did A Stunning Photo Shoot With Their...          1
Filtered Data:
                                                headline  clickbait
4154              male rapper soulmate based zodiac sign          1
23930       9 police officers killed taliban afghanistan          0
15545     15 painful truths mal canadian french speakers          1
31828   report indicates texas state parks still suff...          0
4755    someone redid star wars force awakens trailer...          1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(finaldata['headline'], finaldata['clickbait'],test_size=0.2, random_state=0)

Using TFIDF Vectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text=text.strip()
    return text


Creating the MultinomialNB model

In [ ]:
mnb_tf = MultinomialNB()
mnb_tf.fit(tfidf_train, y_train)
pred = mnb_tf.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
cm = metrics.confusion_matrix(y_test, pred, labels=[1, 0])
print('Confusion Matrix : \n',cm)
matrix = classification_report(y_test,pred,labels=[1,0])
print('Classification report : \n',matrix)

Confusion Matrix : 
 [[3133  112]
 [ 138 3017]]
Classification report : 
               precision    recall  f1-score   support

           1       0.96      0.97      0.96      3245
           0       0.96      0.96      0.96      3155

    accuracy                           0.96      6400
   macro avg       0.96      0.96      0.96      6400
weighted avg       0.96      0.96      0.96      6400



Saving the Model

In [ ]:
pickle.dump(tfidf_vectorizer,open('vectorizer.pkl','wb'))
pickle.dump(mnb_tf, open('clickbait_model.pkl','wb'))

Predicting Using Saved Model

In [ ]:
md_from_joblib = joblib.load('clickbait_model.pkl')
sample="US offers to eliminate duties on Cotton, Africa says it's not enough"
tfidf_vectorizer=joblib.load('vectorizer.pkl')
md_from_joblib.predict(tfidf_vectorizer.transform([wordopt(sample)]))


array([0])